# How to measure and improve Dapper performance

To measure and improve Dapper performance with C#, you can use the following steps:

- Install the Dapper and BenchmarkDotNet in your project.
- Create a class that implements the <code>IBenchmark</code> interface and defines the methods for setting up, executing, and cleaning up the benchmark.
- Create a connection to the AdventureWorks2022 database using the <code>SqlConnection</code> class and the connection string.
- In the execute method, use the <code>Query</code> extension method of Dapper to execute a SQL query and map the results to a C# object. For example, you can query the Product table and map the results to a Product class.
- In the cleanup method, dispose the connection object and release any resources used by the benchmark.
- Use the <code>BenchmarkRunner</code> class to run the benchmark and generate a report that shows the performance metrics such as mean time, standard deviation, allocations, etc.
- To improve the performance of Dapper, you can try some of the following tips:
    * Use parameterized queries instead of string concatenation or interpolation to avoid SQL injection and improve query plan caching.
    * Use the <code>Buffered</code> parameter of the <code>Query</code> method to control whether the results are buffered in memory or streamed from the database. Buffering can improve performance for small result sets, but streaming can reduce memory usage for large result sets.
    * Use the <code>SplitOn</code> parameter of the <code>Query</code> method to specify the column name that separates the objects in a multi-mapping query. This can avoid unnecessary reflection and improve mapping speed.
    * Use the <code>AsNoTracking</code> extension method of EF Core to disable change tracking and improve query performance when you only need to read data from the database.

Here is a C# example of how to use <code>BenchmarkRunner</code> with Dapper. It assumes that you have installed the Dapper and BenchmarkDotNet NuGet packages, and that you have a connection string to the AdventureWorks2022 database. The example benchmarks two methods that query the Product table using different approaches: one using raw SQL and the other using stored procedure. The results are printed to the console and exported to a markdown file.

In [1]:
// Connection string
#load "AppSettings.cs"

In [2]:
#r "nuget:Dapper"
#r "nuget:BenchmarkDotNet"
#r "nuget:Microsoft.Data.SqlClient"

Installed Packages BenchmarkDotNet, 0.13.12 Dapper, 2.1.28 Microsoft.Data.SqlClient, 5.1.5

In [3]:
using System;
using System.Data;
using Microsoft.Data.SqlClient;
using BenchmarkDotNet.Attributes;
using BenchmarkDotNet.Running;
using BenchmarkDotNet.Configs;
using Dapper;

In [4]:
public class Product
{
    public int ProductID { get; set; }
    public string Name { get; set; }
    public decimal ListPrice { get; set; }
}

public class Employee
{
    public int BusinessEntityID { get; set; }
    public int RecursionLevel { get; set; }   
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public string ManagerFirstName { get; set; }
    public string ManagerLastName { get; set; }
}

In [5]:
public class DapperBenchmarks
{   
    private const string sqlQuery = "SELECT ProductID, Name, ListPrice FROM Production.Product WHERE ListPrice > 3000";
    private const string storedProcedure = "uspGetEmployeeManagers";

    [Benchmark]
    public Product[] SqlQuery()
    {
        using (var connection = new SqlConnection(connectionString))
        {
            return connection.Query<Product>(sqlQuery).ToArray();
        }
    }

    [Benchmark]
    public Employee[] StoredProcedure()
    {
        using (var connection = new SqlConnection(connectionString))
        {
            DynamicParameters parameters = new DynamicParameters();
            parameters.Add("@BusinessEntityID", 12);           
            return connection.Query<Employee>(storedProcedure, parameters, commandType: CommandType.StoredProcedure).ToArray();
        }
    }
}

In [6]:
var summary = BenchmarkRunner.Run<DapperBenchmarks>(new DebugInProcessConfig());
Console.WriteLine(summary);

]9;4;3;0\// Validating benchmarks:
//    * No exporters defined, results will not be persisted.

// ***** BenchmarkRunner: Start   *****
// ***** Found 2 benchmark(s) in total *****
// ***** Building 1 exe(s) in Parallel: Start   *****
// ***** Done, took 00:00:00 (0.03 sec)   *****
// Found 2 benchmarks:
//   DapperBenchmarks.SqlQuery: Job-BQHOSZ(Toolchain=InProcessEmitToolchain)
//   DapperBenchmarks.StoredProcedure: Job-BQHOSZ(Toolchain=InProcessEmitToolchain)

Setup power plan (GUID: 8c5e7fda-e8bf-4a96-9a85-a6e23a8c635c FriendlyName: High performance)
// **************************
// Benchmark: DapperBenchmarks.SqlQuery: Job-BQHOSZ(Toolchain=InProcessEmitToolchain)
// *** Execute ***
// Launch: 1 / 1

// Benchmark Process Environment Information:
// BenchmarkDotNet v0.13.12
// Runtime=.NET 8.0.1 (8.0.123.58001), X64 RyuJIT AVX-512F+CD+BW+DQ+VL+VBMI
// GC=Concurrent Server
// HardwareIntrinsics=AVX-512F+CD+BW+DQ+VL+VBMI,AES,BMI1,BMI2,FMA,LZCNT,PCLMUL,POPCNT VectorSize=256
// Job: 